In [1]:
from transformers import Qwen2_5_VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info
import torch

model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2.5-VL-7B-Instruct", torch_dtype="auto", device_map="auto"
)

# default processer
processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-7B-Instruct")

# The default range for the number of visual tokens per image in the model is 4-16384.
# You can set min_pixels and max_pixels according to your needs, such as a token range of 256-1280, to balance performance and cost.
# min_pixels = 256*28*28
# max_pixels = 1280*28*28
# processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-7B-Instruct", min_pixels=min_pixels, max_pixels=max_pixels)

"""messages = [
    # Example 1
    {
        "role": "user",
        "content": [
            {"type": "image", "image": "gqa/2354919.jpg"},
            {"type": "text", "text": "Who is riding a bike? Please provide the bounding box coordinate like [x_min, y_min, x_max, y_max]."}
        ],
    },
    {
        "role": "assistant",
        "content": "The man is riding a bike. The bounding box coordinates are [56, 272, 88, 319]. Reasoning: Identify the bike, associate the person riding it, then name the person."
    },

    # Example 2
    {
        "role": "user",
        "content": [
            {"type": "image", "image": "gqa/2334016.jpg"},
            {"type": "text", "text": "What animal is to the right of the man? Please provide the bounding box coordinate like [x_min, y_min, x_max, y_max]."}
        ],
    },
    {
        "role": "assistant",
        "content": "The animal is a horse. The bounding box coordinates are [339, 161, 419, 237]. Reasoning: Find the man, locate the animal to his right, determine species."
    },

    # Example 3
    {
        "role": "user",
        "content": [
            {"type": "image", "image": "gqa/2415528.jpg"},
            {"type": "text", "text": "The pot is where? Please provide the bounding box coordinate like [x_min, y_min, x_max, y_max]."}
        ],
    },
    {
        "role": "assistant",
        "content": "The pot is on the floor. The bounding box coordinates are [5, 71, 497, 374]. Reasoning: Identify the pot, find its spatial relation, name the location."
    },

    # Example 4
    {
        "role": "user",
        "content": [
            {"type": "image", "image": "gqa/2414884.jpg"},
            {"type": "text", "text": "What is the man watching, a sheep or a donkey? Please provide the bounding box coordinate like [x_min, y_min, x_max, y_max]."}
        ],
    },
    {
        "role": "assistant",
        "content": "The man is watching a sheep. The bounding box coordinates are [299, 124, 310, 135]. Reasoning: Identify the man, see which animal he watches, compare sheep and donkey features."
    },

    # Example 5
    {
        "role": "user",
        "content": [
            {"type": "image", "image": "gqa/2391800.jpg"},
            {"type": "text", "text": "What animal is to the right of the man? Please provide the bounding box coordinate like [x_min, y_min, x_max, y_max]."}
        ],
    },
    {
        "role": "assistant",
        "content": "The animal is a dog. The bounding box coordinates are [169, 82, 250, 182]. Reasoning: Identify the man, locate animal to the right, determine its type."
    },

    # test query
    {
        "role": "user",
        "content": [
            {"type": "image", "image": "gqa/2355243.jpg"},
            {"type": "text", "text": "Who is wearing the backpack? Please provide the bounding box coordinate of the region that can help you answer the question better."}
        ],
    },
]"""


# Preparation for inference
"""text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to("cuda")

# Inference: Generation of the output
generated_ids = model.generate(**inputs, max_new_tokens=100)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
print(output_text)"""


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
You have video processor config saved in `preprocessor.json` file which is deprecated. Video processor configs should be saved in their own `video_preprocessor.json` file. You can rename the file or load and save the processor back which renames it automatically. Loading from `preprocessor.json` will be removed in v5.0.


'text = processor.apply_chat_template(\n    messages, tokenize=False, add_generation_prompt=True\n)\nimage_inputs, video_inputs = process_vision_info(messages)\ninputs = processor(\n    text=[text],\n    images=image_inputs,\n    videos=video_inputs,\n    padding=True,\n    return_tensors="pt",\n)\ninputs = inputs.to("cuda")\n\n# Inference: Generation of the output\ngenerated_ids = model.generate(**inputs, max_new_tokens=100)\ngenerated_ids_trimmed = [\n    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)\n]\noutput_text = processor.batch_decode(\n    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False\n)\nprint(output_text)'

In [2]:
messages = [

    # for printing Bbox coordinates.
    {
        "role": "system", 
        "content": "You must always provide both the final answer and the bounding box coordinates that can help you answer the question better in the format like Answer: (answer).\nCoT BBox: [x_min, y_min, x_max, y_max]. Do not omit either."
    },
    
    # Example 1, for GQA dataset
    {
        "role": "user",
        "content": [
            {"type": "image", "image": "gqa/2354919.jpg"},
            {"type": "text", "text": "Who is riding a bike? Please provide the answer and CoT BBox like [x_min, y_min, x_max, y_max]."}
        ],
    },
    {
        "role": "assistant",
        "content": "Answer: The man is riding a bike.\nCoT BBox: [56, 272, 88, 319]"
    },

    # Example 2
    {
        "role": "user",
        "content": [
            {"type": "image", "image": "gqa/2334016.jpg"},
            {"type": "text", "text": "What animal is to the right of the man? Please provide the answer and CoT BBox like [x_min, y_min, x_max, y_max]."}
        ],
    },
    {
        "role": "assistant",
        "content": "Answer: The animal is a horse.\nCoT BBox: [339, 161, 419, 237]"
    },

    # Example 3
    {
        "role": "user",
        "content": [
            {"type": "image", "image": "gqa/2415528.jpg"},
            {"type": "text", "text": "The pot is where? Please provide the answer and CoT BBox like [x_min, y_min, x_max, y_max]."}
        ],
    },
    {
        "role": "assistant",
        "content": "Answer: The pot is on the floor.\nCoT BBox: [5, 71, 497, 374]"
    },

    # Example 4
    {
        "role": "user",
        "content": [
            {"type": "image", "image": "gqa/2414884.jpg"},
            {"type": "text", "text": "What is the man watching, a sheep or a donkey? Please provide the answer and CoT BBox like [x_min, y_min, x_max, y_max]."}
        ],
    },
    {
        "role": "assistant",
        "content": "Answer: The man is watching a sheep.\nCoT BBox: [299, 124, 310, 135]"
    },

    # Example 5
    {
        "role": "user",
        "content": [
            {"type": "image", "image": "gqa/2391800.jpg"},
            {"type": "text", "text": "What animal is to the right of the man? Please provide the answer and CoT BBox like [x_min, y_min, x_max, y_max]."}
        ],
    },
    {
        "role": "assistant",
        "content": "Answer: The animal is a dog.\nCoT BBox: [169, 82, 250, 182]"
    },

        # Example 6
    {
        "role": "user",
        "content": [
            {"type": "image", "image": "gqa/2355243.jpg"},
            {"type": "text", "text": "Who is wearing the backpack? Please provide the answer and CoT BBox like [x_min, y_min, x_max, y_max]."}
        ],
    },
    {
        "role": "assistant",
        "content": "Answer: The man is wearing a backpack.\nCoT BBox: [301, 117, 436, 325]"
    },

    # Example 7
    {
        "role": "user",
        "content": [
            {"type": "image", "image": "gqa/1159469.jpg"},
            {"type": "text", "text": "What is on the shelf in the top part of the image? Please provide the answer and CoT BBox like [x_min, y_min, x_max, y_max]."}
        ],
    },
    {
        "role": "assistant",
        "content": "Answer: The clock is on the shelf.\nCoT BBox: [582, 55, 610, 86]"
    },

    # test query
    {
        "role": "user",
        "content": [
            {"type": "image", "image": "gqa/2355243.jpg"},
            {"type": "text", "text": "Who is wearing the backpack?"}
        ],
    },
]


# Preparation for inference
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to("cuda")

# Inference: Generation of the output
generated_ids = model.generate(**inputs, max_new_tokens=128)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
print(output_text)


['Answer: The man is wearing a backpack.\nCoT BBox: [301, 117, 436, 325]']


In [2]:
messages = [

    # for printing Bbox coordinates.
    {
        "role": "system", 
        "content": "You must always provide both the final answer and the bounding box coordinates in the format like Answer: (answer).\nCoT BBox: [x_min, y_min, x_max, y_max]. Do not omit either."
    },
    
    # Example 1, for TextVQA dataset.
    {
        "role": "user",
        "content": [
            {"type": "image", "image": "textvqa/11bfb854683ef695.jpg"},
            {"type": "text", "text": "What airline does the plane belong to? Please provide the answer and CoT BBox like [x_min, y_min, x_max, y_max]."}
        ],
    },
    {
        "role": "assistant",
        "content": "Answer: United.\nCoT BBox: [471.44, 409.90, 667.47, 451.13]"
    },

    # Example 2
    {
        "role": "user",
        "content": [
            {"type": "image", "image": "textvqa/0310822ffb3699d4.jpg"},
            {"type": "text", "text": "Which page is the page on the right on? Please provide the answer and CoT BBox like [x_min, y_min, x_max, y_max]."}
        ],
    },
    {
        "role": "assistant",
        "content": "Answer: CLXXXV.\nCoT BBox: [810.46, 54.70, 875.33, 69.64]"
    },

    # Example 3
    {
        "role": "user",
        "content": [
            {"type": "image", "image": "textvqa/6a4acaaebaa143b3.jpg"},
            {"type": "text", "text": "What is the name of the book? Please provide the answer and CoT BBox like [x_min, y_min, x_max, y_max]."}
        ],
    },
    {
        "role": "assistant",
        "content": "Answer: The Saint Goes On.\nCoT BBox: [82.35, 97.57, 258.65, 179.23], [268.06, 96.34, 555.76, 184.81], [117.39, 180.44, 368.78, 256.25], [376.57, 183.18, 517.91, 258.83], [270.29, 439.45, 313.45, 472.28]"
    },

    # Example 4
    {
        "role": "user",
        "content": [
            {"type": "image", "image": "textvqa/a4721119fec66ad6.jpg"},
            {"type": "text", "text": "What is the catchphrase at the bottom of the book? Please provide the answer and CoT BBox like [x_min, y_min, x_max, y_max]."}
        ],
    },
    {
        "role": "assistant",
        "content": "Answer: Flexibility for the future.\nCoT BBox: [58.11, 699.68, 139.24, 725.67], [140.64, 700.64, 174.45, 727.22], [174.66, 700.89, 209.30, 726.53], [210.77, 700.80, 271.83, 729.69]"
    },

    # Example 5
    {
        "role": "user",
        "content": [
            {"type": "image", "image": "textvqa/023232c2030de30a.jpg"},
            {"type": "text", "text": "What brand of marker are the black ones? Please provide the answer and CoT BBox like [x_min, y_min, x_max, y_max]."}
        ],
    },
    {
        "role": "assistant",
        "content": "Answer: Sharpie.\nCoT BBox: [542.52, 236.38, 584.52, 256.95]"
    },

    # test query
    {
        "role": "user",
        "content": [
            {"type": "image", "image": "textvqa/43c80607ec7aac0c.jpg"},
            {"type": "text", "text": "what is the trucks website?"}
        ],
    },
]

# Preparation for inference
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to("cuda")

# Inference: Generation of the output
generated_ids = model.generate(**inputs, max_new_tokens=100)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
print(output_text)


['Answer: kojakitchen.com.\nCoT BBox: [244, 155, 412, 201]']


In [4]:
messages = [

    # for printing Bbox coordinates.
    {
        "role": "system", 
        "content": "You must always provide both the final answer and the bounding box coordinates in the format like Answer: (answer).\nCoT BBox: [x_min, y_min, x_max, y_max]. Do not omit either."
    },
    
    # Example 1, for cub dataset.
    {
        "role": "user",
        "content": [
            {"type": "image", "image": "cub/Painted_Bunting_0071_15209.jpg"},
            {"type": "text", "text": "Does the bird in the picture have purple upperparts and purple underparts? Please provide the answer and CoT BBox like [x_min, y_min, x_max, y_max]."}
        ],
    },
    {
        "role": "assistant",
        "content": "Answer: No.\nCoT BBox: [55.0, 70.0, 389.0, 493.0]"
    },

    # Example 2
    {
        "role": "user",
        "content": [
            {"type": "image", "image": "cub/Ringed_Kingfisher_0108_73169.jpg"},
            {"type": "text", "text": "Does the bird in the picture have black eye and grey forehead? Please provide the answer and CoT BBox like [x_min, y_min, x_max, y_max]."}
        ],
    },
    {
        "role": "assistant",
        "content": "Answer: Yes.\nCoT BBox: [154.0, 123.0, 300.0, 227.0]"
    },

    # Example 3
    {
        "role": "user",
        "content": [
            {"type": "image", "image": "cub/Sayornis_0036_98323.jpg"},
            {"type": "text", "text": "Does the bird in the picture have yellow back and grey leg? Please provide the answer and CoT BBox like [x_min, y_min, x_max, y_max]."}
        ],
    },
    {
        "role": "assistant",
        "content": "Answer: No.\nCoT BBox: [113.0, 153.0, 277.0, 422.0]"
    },

    # Example 4
    {
        "role": "user",
        "content": [
            {"type": "image", "image": "cub/Ring_Billed_Gull_0017_51412.jpg"},
            {"type": "text", "text": "Does the bird in the picture have white throat and white underparts? Please provide the answer and CoT BBox like [x_min, y_min, x_max, y_max]."}
        ],
    },
    {
        "role": "assistant",
        "content": "Answer: Yes.\nCoT BBox: [66.0, 43.0, 290.0, 445.0]"
    },

    # Example 5
    {
        "role": "user",
        "content": [
            {"type": "image", "image": "cub/Scott_Oriole_0005_92362.jpg"},
            {"type": "text", "text": "Does the bird in the picture have grey nape and spotted head? Please provide the answer and CoT BBox like [x_min, y_min, x_max, y_max]."}
        ],
    },
    {
        "role": "assistant",
        "content": "Answer: No.\nCoT BBox: [159.0, 88.0, 300.0, 275.0]"
    },

    # test query
    {
        "role": "user",
        "content": [
            {"type": "image", "image": "cub/Western_Gull_0094_55156.jpg"},
            {"type": "text", "text": "Does the bird in the picture have red wing and orange belly?"}
        ],
    },
]

# Preparation for inference
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to("cuda")

# Inference: Generation of the output
generated_ids = model.generate(**inputs, max_new_tokens=100)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
print(output_text)


['Answer: No.\nCoT BBox: [128, 107, 464, 355]']


In [2]:
messages = [

    # for printing Bbox coordinates.
    {
        "role": "system", 
        "content": "You must always provide both the final answer and the bounding box coordinates in the format like Answer: (answer).\nCoT BBox: [x_min, y_min, x_max, y_max]. Do not omit either."
    },
    
    # Example 1, for DocVQA datasets.
    {
        "role": "user",
        "content": [
            {"type": "image", "image": "docvqa/qnlw0023_6.png"},
            {"type": "text", "text": "What is the program number? Please provide the answer and CoT BBox like [x_min, y_min, x_max, y_max]."}
        ],
    },
    {
        "role": "assistant",
        "content": "Answer: 701269.\nCoT BBox: [252, 721, 487, 751]"
    },

    # Example 2
    {
        "role": "user",
        "content": [
            {"type": "image", "image": "docvqa/jgph0227_6.png"},
            {"type": "text", "text": "What is the title of the document? Please provide the answer and CoT BBox like [x_min, y_min, x_max, y_max]."}
        ],
    },
    {
        "role": "assistant",
        "content": "Answer: WATER ANALYSIS.\nCoT BBox: [582, 197, 1055, 245], [582, 197, 1055, 245]"
    },

    # test query
    {
        "role": "user",
        "content": [
            {"type": "image", "image": "docvqa/nghd0227_12.png"},
            {"type": "text", "text": "What is the Major Research Interest?"}
        ],
    },
]

# Preparation for inference
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to("cuda")

# Inference: Generation of the output
generated_ids = model.generate(**inputs, max_new_tokens=100)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
print(output_text)


['Answer: Cognitive Development.\nCoT BBox: [203, 920, 574, 960]']
